# Basic Queries D1 vary ALL

In [1]:
import time
import statistics as stats
#import os
#os.system('sudo sync; echo 3 > /proc/sys/vm/drop_caches')
start_program = time.time()
import pandas as pd
import time
import statistics as stats
#import os
#os.system('sudo sync; echo 3 > /proc/sys/vm/drop_caches')
import pandas as pd

In [2]:
import random
import time
from tqdm import tqdm
import numpy as np
    
def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))


def random_date(start, end, prop, dform = '%Y-%m-%dT%H:%M:%S'):
    return str_time_prop(start, end, dform, prop)
    
def get_list(elm, n_elm, max_r = 10, prefix = '', suffix = '', apostrophe = True):
    res = ''
    elms = random.sample(range(max_r), n_elm)
    for i in range(n_elm): 
        item = prefix + elm + str(elms[i]) +  suffix 
        if apostrophe: 
            item = "'" + item + "'"
        res += item 
        if i < n_elm - 1: 
            res += ", "
    return res

import math

def percentile(data, perc: int):
    size = len(data)
    return sorted(data)[int(math.ceil((size * perc) / 100)) - 1]


In [3]:
query1, query2, query3, query4, query5 = [{},{}],[{},{}],[{},{}],[{},{}],[{},{}]


In [4]:
# max_duration = {
#     1: 10,
#     2: 15,
#     3: 30,
#     4: 5,
#     5: 75
# }
max_duration = {
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 1
}
rangesUnit = {
    1: "hour",
    2: "hour",
    3: "hour",
    4: "hour",
    5: "hour",
}

n_it = 30


number_stations = 10
number_sensors = 100

def_st = 1
def_s = 10
def_r = 'day'

In [5]:
random.seed(1)

set_st = [str(random.randint(0,9)) for i in range(500)]
set_s = [str(random.randint(0,99)) for i in range(500)]
set_date = [random.random() for i in range(500)]


In [6]:
def to_pm(v):
    return str(round(v[0][0],2)) + "$" + '\\' + "pm$" + str(round(v[1][0],2))


# ClickHouse

In [ ]:
# A simple class
# attribute

c_q1 = """select time, id_station, <sid> FROM d1_same_ts where id_station in <stid>
AND time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
AND time < TIMESTAMP '<timestamp>';"""

# c_q1 = """select time, s<sid> FROM d1_same_ts where id_station='st<stid>'
# AND time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
# AND time < TIMESTAMP '<timestamp>';"""

c_q2 = """select time, id_station, <sid> FROM d1_same_ts where id_station in <stid>
AND time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
AND time < TIMESTAMP '<timestamp>' and <sfilter>""" # s<sid> > 0.95;

c_q3 = """SELECT id_station, <avg_s> FROM d1_same_ts 
WHERE time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
AND time < TIMESTAMP '<timestamp>' 
AND id_station in <stid>
GROUP BY id_station;"""   #avg(s<sid>)

c_q4 = """SELECT id_station, EXTRACT(YEAR FROM time) AS "year",
date_trunc('month', time) AS "month", 
date_trunc('day', time) AS "day", 
date_trunc('hour', time) AS "hour", 
<avg_s> 
FROM d1_same_ts where  time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
AND time < TIMESTAMP '<timestamp>' 
AND id_station in <stid>
GROUP BY id_station, "year", "month", "day", "hour";""" # AVG(s<sid>) AS avg_s<sid>

c_q5 = """select time, id_station, <sid> FROM d1_same_ts where id_station in <stid>
	AND time > TIMESTAMP '<timestamp>' - INTERVAL '<nb>' <rangesUnit> 
	AND time < TIMESTAMP '<timestamp>' ORDER BY
	id_station ASC,
	time ASC WITH FILL STEP 5
	INTERPOLATE ( <sid>, id_station AS id_station );
"""



In [ ]:
from clickhouse_driver import Client
from clickhouse_driver import connect as connect_ClickHouse

class ClickHouse:
 
    # A sample method 
    @staticmethod
    def query(query, max_d, rangesUnit, n_it, n_st = 1, n_s = 10):
#         client = Client('diufrm102')
        conn = connect_ClickHouse("clickhouse://diufrm102")
        cursor = conn.cursor()
#         print(client.execute('SHOW TABLES'))
#         return None, None
        duration = max_d
        results = [[],[]]       
        runtimes = []
        full_time = time.time()
        for i in tqdm(range(n_it)):
#             time.sleep(0.5)
            date = random_date("2019-04-01T00:00:00", "2019-05-01T00:00:00", set_date[(duration*i)%500], dform = '%Y-%m-%dT%H:%M:%S')
            temp = query.replace("<timestamp>", date)
            temp = temp.replace("<nb>", str(duration))
            temp = temp.replace("<rangesUnit>", str(rangesUnit))
#             temp = temp.replace("<sid>", str(set_s[(duration*i)%500]))
            # st
            li = ['st' + str(z) for z in random.sample(range(number_stations), n_st)]
            q = '(' + "'" + li[0] + "'"
            for j in li[1:]:
                q += ', ' + "'" + j + "'"
            q += ")"
            temp = temp.replace("<stid>", q)
            
            # sensor
            li = ['s' + str(z) for z in random.sample(range(number_sensors), n_s)]
            q = li[0]
            q_filter = '(' + li[0] + ' > 0.95'
            q_avg = 'avg(' + li[0] + ')'
            for j in li[1:]:
                q += ', ' + j
                q_filter += ' OR ' + j + ' > 0.95'
                q_avg += ', ' + 'avg(' + j + ')'
            temp = temp.replace("<sid>", q)
            temp = temp.replace("<sfilter>", q_filter + ')')
            temp = temp.replace("<avg_s>", q_avg)
            

            start = time.time()
#             print(temp)
            cursor.execute(temp)
            print(len(cursor.fetchall()))
            diff = (time.time()-start)*1000
#                 print(temp, diff)
            runtimes.append(diff)
            if time.time() - full_time > 60 * 10 and i > 3: 
                break              
#             print(temp)
        results[0].append(stats.mean(runtimes))
#             results[1].append(percentile(runtimes,85))
        results[1].append(stats.stdev(runtimes))
        conn.close()
        return results[0], results[1]

In [ ]:
for st in range(2, 11, 2):
# for s in range(10, 101, 10):
# for r in ["minute", "hour", "day", "week", "month"]:
    print(ClickHouse.query(c_q5, 1, "hour", 2, n_st = st, n_s = 10))
# print(ClickHouse.query(c_q2, 1, "hour", 2, n_st = 2, n_s = 10))
# print(ClickHouse.query(c_q2, 1, "hour", 2, n_st = 2, n_s = 100))


# eXtremeDB

In [30]:
# A simple class
# d1_same_ts_v
# attribute
# e_q1 = """select seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, s<sid>@tt FROM d1_v WHERE id_station = 'st<stid>';"""
e_q1 = """select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <sid> FROM d1_v WHERE id_station in <stid>;"""


e_q2 = """select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <sfilter> as fe, <sid_filtered> FROM d1_v WHERE id_station in <stid>; """ #!seq_filter_search(s<sid>@tt > 0.95, tt),. <sid>@fe 

e_q2AND = """select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <sfilterAND> as fe, <sid_filtered> FROM d1_v WHERE id_station in <stid>; """ #!seq_filter_search(s<sid>@tt > 0.95, tt),. <sid>@fe 


e_q3 = """SELECT id_station, ! seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <seq_avg> FROM d1_v WHERE id_station in <stid>;""" # where id_station = 'st<stid>', <seq_avg>(<sid>@tt)


e_q4 = """select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>, <timestamp>) as tt, t@tt/3600 as hour, <seq_group_agg_dev> FROM d1_v where id_station in <stid>""" #seq_group_agg_dev(s<sid>@tt, t@tt/3600) 


e_q5 = """select seq_aprogres_datetime(<timestamp> -  <nb> * <rangesUnit>, 5, <nb> * <rangesUnit>) as ts5, <seq_stretch> from d1_v where id_station in <stid>;""" #seq_stretch(ts5,t,s<sid>)



In [33]:
class EXtremeDB:
 
    # A sample method 
    @staticmethod
    def query(query, max_d, rangesUnit, n_it, n_st = 1, n_s = 10):
        # map the inputs to the function blocks
        import exdb 
        import datetime
        exdb.init_runtime(debug = False, shm = False, disk = False, tmgr = 'mursiw')
        con = exdb.connect('diufrm118', 5001)
        curs = con.cursor()
        curs.execute("SELECT s23 FROM d1_same_ts_v where id_station = 'st3'")
        curs.fetchall()
        results = [[],[]]
        options = {"day" : 60 * 60* 24,
                   "week" : 60 * 60* 24 * 7,
                   "minute" : 60,
                   "hour" : 60 * 60,
                   "second" : 1,
                   "month" : 60 * 60 * 24 * 30,
                   "year" :  60 * 60 * 24 * 30 * 12
        }
        duration = max_d
        runtimes = []
        full_time = time.time()
        for i in tqdm(range(n_it)):
#             time.sleep(0.5)
            date = random_date("2019-04-01 00:00:00", "2019-05-01 00:00:00", set_date[(duration*i)%500], dform = '%Y-%m-%d %H:%M:%S')
            date = int(time.mktime(datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S').timetuple()))
            temp = query.replace("<timestamp>", str(date))
            temp = temp.replace("<nb>", str(duration))
#             temp = temp.replace("<sid>", str(set_s[(duration*i)%500]))
            temp = temp.replace("<rangesUnit>", str(options[rangesUnit]))
        
            li = ['st' + str(z) for z in random.sample(range(number_stations), n_st)]
            q = '(' + "'" + li[0] + "'"
            for j in li[1:]:
                q += ', ' + "'" + j + "'"
            q += ")"
            temp = temp.replace("<stid>", q)
            
            # sensor
         
            
            
            li = ['s' + str(z) + "@tt" for z in random.sample(range(number_sensors), n_s)]
            li_filtered = ['s' + str(z) + "@fe" for z in random.sample(range(number_sensors), n_s)]
#             <seq_group_agg_dev> seq_group_agg_dev(s<sid>@tt, t@tt/3600), 
#             <seq_stretch> #seq_stretch(ts5,t,s<sid>), 
#             <sfilter> !seq_filter_search(s<sid>@tt > 0.95, tt)
            
            q = li[0]
            q_filtered = li_filtered[0]
            q_seq_group_agg_dev = "seq_group_agg_dev(" + li[0] + " , t@tt/3600)" 
            q_seq_avg = "seq_avg(" + li[0] + ")" 
            q_seq_stretch = "seq_stretch(ts5,t," + li[0].split('@')[0] + ")" 
#             q_filter = "!seq_filter_search(" +li[0] + "> 0.95, tt)"
            q_filter = "!seq_filter_search(" +li[0] + "> 0.01"
            q_filterAND = "!seq_filter_search(" +li[0] + "> 0.01"
            
#             q_avg = 'avg(' + li[0] + ')'
            for j in li_filtered[1:]:
                q_filtered += ', ' + j
        
            for j in li[1:]:
                q += ', ' + j
                q_seq_avg += ", seq_avg(" + j + ")" 
                q_seq_group_agg_dev += ", seq_group_agg_dev(" + j + " , t@tt/3600)" #        li[0] + ' > 0.95'
                q_seq_stretch += ", seq_stretch(ts5,t," + j.split('@')[0] + ")" 
#                 q_filter += ", !seq_filter_search(" + j + "> 0.95, tt)"
                q_filter += " OR " + j + "> 0.01"
                q_filterAND += " AND " + j + "> 0.01"
#                 q_filter += ' OR ' + j + ' > 0.95'
#                 q_avg += ', ' + 'avg(' + j + ')'
            temp = temp.replace("<sid>", q)
            temp = temp.replace("<seq_avg>", q_seq_avg)
            temp = temp.replace("<sid_filtered>", q_filtered )
            temp = temp.replace("<seq_group_agg_dev>", q_seq_group_agg_dev)
            temp = temp.replace("<sfilter>", q_filter + ", tt)")
            temp = temp.replace("<sfilterAND>", q_filterAND + ", tt)")
            temp = temp.replace("<seq_stretch>", q_seq_stretch)
#             temp = temp.replace("<avg_s>", q_avg)            
            print(temp)
#             print(temp)
            start = time.time()
            curs.execute(temp)
            # print(curs.rowcount)
            curs.fetchall()
            diff = (time.time()-start)*1000
#                 print(temp, diff)
            runtimes.append(diff)
            if time.time() - full_time > 60 * 10 and i > 3: 
                break                
#         print(rxuntimes)
        results[0].append(stats.mean(runtimes))
#             results[1].append(percentile(runtimes,85))
        results[1].append(stats.stdev(runtimes))
        con.close()
        return results


In [34]:
# for st in range(2, 11, 2):
# for s in range(10, 101, 10):
for r in ["minute", "hour", "day", "week", "month"]:
    print(EXtremeDB.query(e_q4, 1, r, 2, n_st = 1, n_s = 10))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 476.46it/s]


select id_station, seq_search(t,1556461234 - 1 * 60, 1556461234) as tt, t@tt/3600 as hour, seq_group_agg_dev(s65@tt , t@tt/3600), seq_group_agg_dev(s81@tt , t@tt/3600), seq_group_agg_dev(s8@tt , t@tt/3600), seq_group_agg_dev(s24@tt , t@tt/3600), seq_group_agg_dev(s42@tt , t@tt/3600), seq_group_agg_dev(s77@tt , t@tt/3600), seq_group_agg_dev(s59@tt , t@tt/3600), seq_group_agg_dev(s74@tt , t@tt/3600), seq_group_agg_dev(s23@tt , t@tt/3600), seq_group_agg_dev(s53@tt , t@tt/3600) FROM d1_v where id_station in ('st7')
select id_station, seq_search(t,1554258458 - 1 * 60, 1554258458) as tt, t@tt/3600 as hour, seq_group_agg_dev(s87@tt , t@tt/3600), seq_group_agg_dev(s55@tt , t@tt/3600), seq_group_agg_dev(s29@tt , t@tt/3600), seq_group_agg_dev(s22@tt , t@tt/3600), seq_group_agg_dev(s10@tt , t@tt/3600), seq_group_agg_dev(s6@tt , t@tt/3600), seq_group_agg_dev(s16@tt , t@tt/3600), seq_group_agg_dev(s91@tt , t@tt/3600), seq_group_agg_dev(s33@tt , t@tt/3600), seq_group_agg_dev(s54@tt , t@tt/3600) FROM

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 305.45it/s]


select id_station, seq_search(t,1556461234 - 1 * 3600, 1556461234) as tt, t@tt/3600 as hour, seq_group_agg_dev(s50@tt , t@tt/3600), seq_group_agg_dev(s85@tt , t@tt/3600), seq_group_agg_dev(s5@tt , t@tt/3600), seq_group_agg_dev(s22@tt , t@tt/3600), seq_group_agg_dev(s73@tt , t@tt/3600), seq_group_agg_dev(s82@tt , t@tt/3600), seq_group_agg_dev(s42@tt , t@tt/3600), seq_group_agg_dev(s35@tt , t@tt/3600), seq_group_agg_dev(s77@tt , t@tt/3600), seq_group_agg_dev(s6@tt , t@tt/3600) FROM d1_v where id_station in ('st1')
select id_station, seq_search(t,1554258458 - 1 * 3600, 1554258458) as tt, t@tt/3600 as hour, seq_group_agg_dev(s67@tt , t@tt/3600), seq_group_agg_dev(s33@tt , t@tt/3600), seq_group_agg_dev(s54@tt , t@tt/3600), seq_group_agg_dev(s15@tt , t@tt/3600), seq_group_agg_dev(s43@tt , t@tt/3600), seq_group_agg_dev(s72@tt , t@tt/3600), seq_group_agg_dev(s93@tt , t@tt/3600), seq_group_agg_dev(s97@tt , t@tt/3600), seq_group_agg_dev(s30@tt , t@tt/3600), seq_group_agg_dev(s0@tt , t@tt/3600) F

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 30.38it/s]


...

select id_station, seq_search(t,1556461234 - 1 * 2592000, 1556461234) as tt, t@tt/3600 as hour, seq_group_agg_dev(s70@tt , t@tt/3600), seq_group_agg_dev(s66@tt , t@tt/3600), seq_group_agg_dev(s59@tt , t@tt/3600), seq_group_agg_dev(s89@tt , t@tt/3600), seq_group_agg_dev(s18@tt , t@tt/3600), seq_group_agg_dev(s71@tt , t@tt/3600), seq_group_agg_dev(s6@tt , t@tt/3600), seq_group_agg_dev(s84@tt , t@tt/3600), seq_group_agg_dev(s81@tt , t@tt/3600), seq_group_agg_dev(s31@tt , t@tt/3600) FROM d1_v where id_station in ('st7')


 50%|██████████████████████████████████████████████████████████                                                          | 1/2 [00:00<00:00,  1.35it/s]

select id_station, seq_search(t,1554258458 - 1 * 2592000, 1554258458) as tt, t@tt/3600 as hour, seq_group_agg_dev(s17@tt , t@tt/3600), seq_group_agg_dev(s82@tt , t@tt/3600), seq_group_agg_dev(s24@tt , t@tt/3600), seq_group_agg_dev(s48@tt , t@tt/3600), seq_group_agg_dev(s59@tt , t@tt/3600), seq_group_agg_dev(s64@tt , t@tt/3600), seq_group_agg_dev(s5@tt , t@tt/3600), seq_group_agg_dev(s67@tt , t@tt/3600), seq_group_agg_dev(s97@tt , t@tt/3600), seq_group_agg_dev(s56@tt , t@tt/3600) FROM d1_v where id_station in ('st5')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.40it/s]

[[713.6216163635254], [34.729677522552855]]


# Run all 

In [ ]:
print(to_pm(ClickHouse.query(c_q1, 7, "day", 5, n_st = 1, n_s = 10)))


In [ ]:
print(to_pm(EXtremeDB.query(e_q1, 7, "day", 5, n_st = 1, n_s = 10)))

In [ ]:
results = [{} for i in range(5)]

for i in range(5):
    for sys in ['clickhouse', 'extremedb']:
        results[i][sys] = []

In [ ]:
[1] + [i for i in range(10, 101, 10)]


In [28]:
def queryClickHouse():
    for i in range(5):
        results[i]["clickhouse"] = []
    for n_st in [1] + [i for i in range(2, 11, 2)]:
        results[0]['clickhouse'].append(to_pm(ClickHouse.query(c_q1, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
        results[1]['clickhouse'].append(to_pm(ClickHouse.query(c_q2, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
#         results[2]['clickhouse'].append(to_pm(ClickHouse.query(c_q3, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
#         results[3]['clickhouse'].append(to_pm(ClickHouse.query(c_q4, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
#         results[4]['clickhouse'].append(to_pm(ClickHouse.query(c_q5, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
    for n_s in [1] + [i for i in range(10, 101, 10)]:
        results[0]['clickhouse'].append(to_pm(ClickHouse.query(c_q1, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
        results[1]['clickhouse'].append(to_pm(ClickHouse.query(c_q2, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
#         results[2]['clickhouse'].append(to_pm(ClickHouse.query(c_q3, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
#         results[3]['clickhouse'].append(to_pm(ClickHouse.query(c_q4, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
#         results[4]['clickhouse'].append(to_pm(ClickHouse.query(c_q5, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
    for r in ['minute', 'hour', 'day', 'week', 'month']:
        results[0]['clickhouse'].append(to_pm(ClickHouse.query(c_q1, 1, r, n_it, n_st = def_st, n_s = def_s)))
        results[1]['clickhouse'].append(to_pm(ClickHouse.query(c_q2, 1, r, n_it, n_st = def_st, n_s = def_s)))
#         results[2]['clickhouse'].append(to_pm(ClickHouse.query(c_q3, 1, r, n_it, n_st = def_st, n_s = def_s)))
#         results[3]['clickhouse'].append(to_pm(ClickHouse.query(c_q4, 1, r, n_it, n_st = def_st, n_s = def_s)))
#         results[4]['clickhouse'].append(to_pm(ClickHouse.query(c_q5, 1, r, n_it, n_st = def_st, n_s = def_s)))

def queryExtremeDB():    
    for i in range(5):
        results = []    
    for n_st in [1] + [i for i in range(2, 11, 2)]:
#         results[0]['extremedb'].append(to_pm(EXtremeDB.query(e_q1, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
        results.append(to_pm(EXtremeDB.query(e_q2, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
#         results[2]['extremedb'].append(to_pm(EXtremeDB.query(e_q3, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
#         results[3]['extremedb'].append(to_pm(EXtremeDB.query(e_q4, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
#         results[4]['extremedb'].append(to_pm(EXtremeDB.query(e_q5, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
    results.append(-1)
    for n_s in [1] + [i for i in range(10, 101, 10)]:
#         results[0]['extremedb'].append(to_pm(EXtremeDB.query(e_q1, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
        results.append(to_pm(EXtremeDB.query(e_q2, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
#         results[2]['extremedb'].append(to_pm(EXtremeDB.query(e_q3, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
#         results[3]['extremedb'].append(to_pm(EXtremeDB.query(e_q4, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
#         results[4]['extremedb'].append(to_pm(EXtremeDB.query(e_q5, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
    results.append(-1)
    for r in ['minute', 'hour', 'day', 'week', 'month']:
#         results[0]['extremedb'].append(to_pm(EXtremeDB.query(e_q1, 1, r, n_it, n_st = def_st, n_s = def_s)))
        results.append(to_pm(EXtremeDB.query(e_q2, 1, r, n_it, n_st = def_st, n_s = def_s)))
#         results[2]['extremedb'].append(to_pm(EXtremeDB.query(e_q3, 1, r, n_it, n_st = def_st, n_s = def_s)))
#         results[3]['extremedb'].append(to_pm(EXtremeDB.query(e_q4, 1, r, n_it, n_st = def_st, n_s = def_s)))
#         results[4]['extremedb'].append(to_pm(EXtremeDB.query(e_q5, 1, r, n_it, n_st = def_st, n_s = def_s)))
    return results


def queryAll():
    queryClickHouse()
    queryExtremeDB()

In [ ]:
# TimescaleDB.query(t_q5, 7, "day", 100, n_st = 1, n_s = 10)

In [ ]:
# Influx_Wide.query(iw_q5, 1, 'month', 5, n_st = def_st, n_s = def_s)

In [ ]:
start_program = time.time()
# results = queryAll(duration_range)

In [ ]:
queryClickHouse()


In [31]:
queryExtremeDB()


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [01:22<00:00,  2.74s/it]


['97.93$\\pm$3.14',
 '185.32$\\pm$5.16',
 '364.07$\\pm$14.16',
 '540.41$\\pm$17.75',
 '711.78$\\pm$26.6',
 '883.39$\\pm$40.34',
 -1,
 '19.07$\\pm$1.19',
 '97.43$\\pm$3.28',
 '199.15$\\pm$8.36',
 '329.54$\\pm$24.6',
 '491.12$\\pm$42.86',
 '687.95$\\pm$71.53',
 '1000.8$\\pm$109.81',
 '1098.19$\\pm$148.24',
 '1467.53$\\pm$135.51',
 '1635.75$\\pm$199.08',
 '1824.95$\\pm$220.07',
 -1,
 '2.13$\\pm$0.56',
 '6.3$\\pm$1.03',
 '96.05$\\pm$2.97',
 '623.05$\\pm$17.64',
 '2737.61$\\pm$121.77']

In [ ]:
# results = queryAllClickHouse(duration_range)
# results = queryAllClickHouse(duration_range)
stop_program = time.time() 

In [ ]:
print('Benchmark Runtime: %s minutes' % str((stop_program - start_program)/60))

In [ ]:
print(results[4]['timescaledb'])

results[0] = { k: results[0][k] for k in ["clickhouse","druid_wide","extremedb","influx", "influx_wide", "monetdb","questdb","timescaledb"] }
results[1] = { k: results[1][k] for k in ["clickhouse","druid_wide","extremedb","influx", "influx_wide", "monetdb","questdb","timescaledb"] }
results[2] = { k: results[2][k] for k in ["clickhouse","druid_wide","extremedb","influx", "influx_wide", "monetdb","questdb","timescaledb"] }
results[3] = { k: results[3][k] for k in ["clickhouse","druid_wide","extremedb","influx", "influx_wide", "monetdb","questdb","timescaledb"] }
results[4] = { k: results[4][k] for k in ['extremedb', "influx", "influx_wide", 'questdb', 'timescaledb'] }


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
import matplotlib

# colors = {'clickhouse': "#584A9D", 'druid': "#CF3650", 'extremedb': "#9966CC", 'influx': "green", 'monetdb': "#915C83", 'questdb': "#3D2B1F", 'timescaledb': "orange"}
for i in tqdm(range(len(results))): 
    print('query ', i+1)
    font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

    matplotlib.rc('font', **font)    
    
    plt.figure(figsize=(30,6))
    plt.subplot(1, 3, 1)
    plt.suptitle('Query ' + str(i+1) +  '; default: Station = 1, Sensor = 10, Range = hour;')


    df_all = pd.DataFrame(results[i])
    df_all.index = [1] + [i for i in range(2, 11, 2)] + [1] + [i for i in range(10, 101, 10)] + ["minute", "hour", "day", "week", "month"]
#     df_all.style.highlight_max(color = 'lightgreen', axis = 1)
    print(df_all)
#     print('problematic: ')
#     for col in df_all: 
#         temp = df_all[df_all[col].str.split('$').str[2].astype(float) / df_all[col].str.split('$').str[0].astype(float) > 0.3]
#         if not temp.empty: print(temp, col)
    
    df = df_all[:6]
#     df.index = [1] + [i for i in range(2, 11, 2)]
    df_runtime = df.copy()
    df_variance = df.copy()
    for col in df: 
        df_runtime[col] = df[col].str.split('$').str[0].astype(float)
        df_variance[col] = df[col].str.split('$').str[2].astype(float)
#         df_runtime[col].plot(logy = True, marker='o', logx = False, xticks = df_runtime.index, title = "Query " + str(i+1), xlabel = "# Stations", ylabel = "Runtime (ms)")
        plt.errorbar(df_runtime[col].index, df_runtime[col], linewidth=4, elinewidth=2, markeredgewidth=2, yerr=df_variance[col], marker='o') #, color = colors[col])
    plt.yscale('log')
#     plt.title("Query " + str(i+1))
    plt.ylabel("Runtime (ms)")
    plt.xlabel("# Stations")
#     plt.legend(['clickhouse', 'druid', 'extremedb', 'influx', 'monetdb', 'questdb', 'timescaledb'], loc='upper left', bbox_to_anchor=(1.05, 1))

    print()

    plt.subplot(1, 3, 2)
    df = df_all[6:17]
#     df.index = [1] + [i for i in range(10, 101, 10)]
    df_runtime = df.copy()
    df_variance = df.copy()
    for col in df: 
        df_runtime[col] = df[col].str.split('$').str[0].astype(float)
        df_variance[col] = df[col].str.split('$').str[2].astype(float)
#         df_runtime[col].plot(logy = True, marker='o', logx = False, xticks = df_runtime.index, title = "Query " + str(i+1), xlabel = "# Stations", ylabel = "Runtime (ms)")
        plt.errorbar(df_runtime[col].index, df_runtime[col], linewidth=4, elinewidth=2, markeredgewidth=2, yerr=df_variance[col], marker='o') #, color = colors[col])
    plt.yscale('log')
#     plt.xscale('log')
#     plt.title("Query " + str(i+1))
    plt.ylabel("Runtime (ms)")
    plt.xlabel("# Sensors")
#     plt.legend(['clickhouse', 'druid', 'extremedb', 'influx', 'monetdb', 'questdb', 'timescaledb'], loc='upper left', bbox_to_anchor=(1.05, 1))

    print()
    
    
    plt.subplot(1, 3, 3)
    df = df_all[17:22]
#     df.index = ["minute", "hour", "day", "week", "month"]
    df_runtime = df.copy()
    df_variance = df.copy()
    for col in df: 
        df_runtime[col] = df[col].str.split('$').str[0].astype(float)
        df_variance[col] = df[col].str.split('$').str[2].astype(float)
#         df_runtime[col].plot(logy = True, marker='o', logx = False, xticks = df_runtime.index, title = "Query " + str(i+1), xlabel = "# Stations", ylabel = "Runtime (ms)")
        plt.errorbar(df_runtime[col].index, df_runtime[col], linewidth=4, elinewidth=2, markeredgewidth=2, yerr=df_variance[col], marker='o') #, color = colors[col])
    plt.xticks(range(0,len(df_runtime.index)), df_runtime.index)
    plt.yscale('log')
#     plt.title("Query " + str(i+1))
    plt.ylabel("Runtime (ms)")
    plt.xlabel("Range")
    plt.legend(['clickhouse', 'druid', 'extremedb', 'influx', 'monetdb', 'questdb', 'timescaledb'], loc='upper left', bbox_to_anchor=(0.9, 1))
    plt.savefig('query' + str(i+1) +'.pdf', dpi=400)
    plt.show()

    print()
#     print('query', i+1)
#     df = pd.DataFrame(results[i])[3:6]
#     df.index = [1,10,100]
#     df_runtime = df.copy()
#     df_variance = df.copy()
#     for col in df: 
#         df_runtime[col] = df[col].str.split('$').str[0].astype(float)
#         df_variance[col] = df[col].str.split('$').str[2].astype(float)
#     print(df_runtime)
#     df_runtime.plot(logy = True, marker='o', logx = True, xticks = df_runtime.index, title = "Query " + str(i+1), xlabel = "# Sensors", ylabel = "Runtime (ms)")
#     print(df_variance)
#     print()

#     print('query', i+1)
#     df = pd.DataFrame(results[i])[6:]
#     df.index = ["minute", "hour", "day", "week", "month"]
#     df_runtime = df.copy()
#     df_variance = df.copy()
#     for col in df: 
#         df_runtime[col] = df[col].str.split('$').str[0].astype(float)
#         df_variance[col] = df[col].str.split('$').str[2].astype(float)
#     print(df_runtime)
#     df_runtime.plot(logy = True, marker='o', title = "Query " + str(i+1), xlabel = "Range", ylabel = "Runtime (ms)")
#     plt.xticks(range(0,len(df_runtime.index)), df_runtime.index)
#     print(df_variance)
#     print()

                
    
    


In [ ]:
from functools import partial

def to_latex(df):
    cols = df.columns
    rows = []
    for index, row in df.iterrows():
        l = row.tolist()
#         for i in range(len(l)): 
#             if l[i]: 
#                 l[i] = float(l[i].split('$')[0])
        l_sorted = sorted(l, key=lambda x: float('inf') if x is None else float(x.split('$')[0]))
        l[l.index(l_sorted[0])] = '\\textbf{'+ str(l[l.index(l_sorted[0])]) + '$^\\star$}'
        l[l.index(l_sorted[1])] = '\\textbf{'+ str(l[l.index(l_sorted[1])]) + '$^\\bullet$}'
#         print(l)
        rows.append(l)
#     print(rows)
    df = pd.DataFrame(np.array(rows),
                   columns=cols, index= df.index)  
    return df
    return df.to_latex(index=False,
            header=cols,
            escape=False)                   


In [ ]:
for i in tqdm(range(len(results))): 
    print('query ', i+1)
    df_all = pd.DataFrame(results[i])
    df_all.index = [1] + [i for i in range(2, 11, 2)] + [1] + [i for i in range(10, 101, 10)] + ["minute", "hour", "day", "week", "month"]
    print(to_latex(df_all))
    print()
    
    
    

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
import matplotlib

systems_to_plot = ['clickhouse', 'timescaledb']

# colors = {'clickhouse': "#584A9D", 'druid': "#CF3650", 'extremedb': "#9966CC", 'influx': "green", 'monetdb': "#915C83", 'questdb': "#3D2B1F", 'timescaledb': "orange"}
for i in tqdm(range(len(results))): 
    print('query ', i+1)
    font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

    matplotlib.rc('font', **font)    
    
    plt.figure(figsize=(30,6))
    plt.subplot(1, 3, 1)
    plt.suptitle('Query ' + str(i+1) +  '; default: Station = 1, Sensor = 10, Range = hour;')


    df_all = pd.DataFrame(results[i])
    df_all.index = [1] + [i for i in range(2, 11, 2)] + [1] + [i for i in range(10, 101, 10)] + ["minute", "hour", "day", "week", "month"]
#     df_all.style.highlight_max(color = 'lightgreen', axis = 1)
#     print(df_all)
#     print('problematic: ')
#     for col in df_all: 
#         temp = df_all[df_all[col].str.split('$').str[2].astype(float) / df_all[col].str.split('$').str[0].astype(float) > 0.3]
#         if not temp.empty: print(temp, col)
    
    df = df_all[:6]
#     df.index = [1] + [i for i in range(2, 11, 2)]
    df_runtime = df.copy()
    df_variance = df.copy()
    for col in systems_to_plot: 
        df_runtime[col] = df[col].str.split('$').str[0].astype(float)
        df_variance[col] = df[col].str.split('$').str[2].astype(float)
#         df_runtime[col].plot(logy = True, marker='o', logx = False, xticks = df_runtime.index, title = "Query " + str(i+1), xlabel = "# Stations", ylabel = "Runtime (ms)")
        plt.errorbar(df_runtime[col].index, df_runtime[col], linewidth=4, elinewidth=2, markeredgewidth=2, yerr=df_variance[col], marker='o') #, color = colors[col])
#     plt.yscale('log')
#     plt.title("Query " + str(i+1))
    plt.ylabel("Runtime (ms)")
    plt.xlabel("# Stations")
#     plt.legend(['clickhouse', 'druid', 'extremedb', 'influx', 'monetdb', 'questdb', 'timescaledb'], loc='upper left', bbox_to_anchor=(1.05, 1))

    print()

    plt.subplot(1, 3, 2)
    df = df_all[6:17]
#     df.index = [1] + [i for i in range(10, 101, 10)]
    df_runtime = df.copy()
    df_variance = df.copy()
    for col in systems_to_plot: 
        df_runtime[col] = df[col].str.split('$').str[0].astype(float)
        df_variance[col] = df[col].str.split('$').str[2].astype(float)
#         df_runtime[col].plot(logy = True, marker='o', logx = False, xticks = df_runtime.index, title = "Query " + str(i+1), xlabel = "# Stations", ylabel = "Runtime (ms)")
        plt.errorbar(df_runtime[col].index, df_runtime[col], linewidth=4, elinewidth=2, markeredgewidth=2, yerr=df_variance[col], marker='o') #, color = colors[col])
#     plt.yscale('log')
#     plt.xscale('log')
#     plt.title("Query " + str(i+1))
    plt.ylabel("Runtime (ms)")
    plt.xlabel("# Sensors")
#     plt.legend(['clickhouse', 'druid', 'extremedb', 'influx', 'monetdb', 'questdb', 'timescaledb'], loc='upper left', bbox_to_anchor=(1.05, 1))

    print()
    
    
    plt.subplot(1, 3, 3)
    df = df_all[17:22]
#     df.index = ["minute", "hour", "day", "week", "month"]
    df_runtime = df.copy()
    df_variance = df.copy()
    for col in systems_to_plot: 
        df_runtime[col] = df[col].str.split('$').str[0].astype(float)
        df_variance[col] = df[col].str.split('$').str[2].astype(float)
#         df_runtime[col].plot(logy = True, marker='o', logx = False, xticks = df_runtime.index, title = "Query " + str(i+1), xlabel = "# Stations", ylabel = "Runtime (ms)")
        plt.errorbar(df_runtime[col].index, df_runtime[col], linewidth=4, elinewidth=2, markeredgewidth=2, yerr=df_variance[col], marker='o') #, color = colors[col])
    plt.xticks(range(0,len(df_runtime.index)), df_runtime.index)
#     plt.yscale('log')
#     plt.title("Query " + str(i+1))
    plt.ylabel("Runtime (ms)")
    plt.xlabel("Range")
    plt.legend(systems_to_plot, loc='upper left', bbox_to_anchor=(0.9, 1))
    plt.savefig('query' + str(i+1) +'.pdf', dpi=400)
    plt.show()

    print()
#     print('query', i+1)
#     df = pd.DataFrame(results[i])[3:6]
#     df.index = [1,10,100]
#     df_runtime = df.copy()
#     df_variance = df.copy()
#     for col in df: 
#         df_runtime[col] = df[col].str.split('$').str[0].astype(float)
#         df_variance[col] = df[col].str.split('$').str[2].astype(float)
#     print(df_runtime)
#     df_runtime.plot(logy = True, marker='o', logx = True, xticks = df_runtime.index, title = "Query " + str(i+1), xlabel = "# Sensors", ylabel = "Runtime (ms)")
#     print(df_variance)
#     print()

#     print('query', i+1)
#     df = pd.DataFrame(results[i])[6:]
#     df.index = ["minute", "hour", "day", "week", "month"]
#     df_runtime = df.copy()
#     df_variance = df.copy()
#     for col in df: 
#         df_runtime[col] = df[col].str.split('$').str[0].astype(float)
#         df_variance[col] = df[col].str.split('$').str[2].astype(float)
#     print(df_runtime)
#     df_runtime.plot(logy = True, marker='o', title = "Query " + str(i+1), xlabel = "Range", ylabel = "Runtime (ms)")
#     plt.xticks(range(0,len(df_runtime.index)), df_runtime.index)
#     print(df_variance)
#     print()

                
    
    


In [ ]:
results[4]['influx']

results[4] = { k: results[4][k] for k in ['extremedb', 'influx', "influx_wide", 'questdb', 'timescaledb'] }


print()

In [ ]:
results[1]["clickhouse"]

In [ ]:
results[1]["clickhouse"] = []
for n_st in [1] + [i for i in range(2, 11, 2)]:
    results[1]['clickhouse'].append(to_pm(ClickHouse.query(c_q2, 1, def_r, n_it, n_st = n_st, n_s = def_s)))
for n_s in [1] + [i for i in range(10, 101, 10)]:
    results[1]['clickhouse'].append(to_pm(ClickHouse.query(c_q2, 1, def_r, n_it, n_st = def_st, n_s = n_s)))
for r in ['minute', 'hour', 'day', 'week', 'month']:
    results[1]['clickhouse'].append(to_pm(ClickHouse.query(c_q2, 1, r, n_it, n_st = def_st, n_s = def_s)))
